In [ ]:
using Pkg
Pkg.add("IJulia")
Pkg.add("XLSX")
Pkg.add("Oscar")
import XLSX
using Oscar

Next, we need to build our algebraic structure, i.e., our Polynomial Ring.
We map the variable R1 to an object of abstract type residue ring of integers modulo 2, i.e. {0,1}.
Then, we construct a multivariate polynomial ring myR in (A, B, C, D, E, F, G) over integers.
The vector of strings ["C", "AH", "PD", "G", "L", "P", "D"] as argument of the PolynomialRing constructor defines how our generating values A, B, C, D, E, F, G should be printed. We will later select the object properties matching these strings (i.e. "C" = complete; "AH" = amputated head; ...) from the excel sheet.
We assign the list of atoms (generator variables) in myR to a variable myvars.

? can i Use "B" like in the text, instead of "myR"? no bcuz b is the boolean alg and we dont have the bool eigenschaft yet

In [ ]:
R1 = ResidueRing(ZZ, 2);
myR, (A, B, C, D, E, F, G, H, I, J, K, L, M, N, O, P) = PolynomialRing(R1, ["C", "A", "AH", "MH", "PA", "PD", "LD", "DI", "F", "G", "L", "S", "SI", "P", "R", "D"]);
myvars=[A, B, C, D, E, F, G, H, I, J, K, L, M, N, O, P];

From the excel file named "Cachette_Matrix_fuer_Noor.xlsx", we extract the information about each object in the Cachette. By information we mean, that for each object, we get a list of values in {0,1}, with 0 meaning the object does not have a specific object property and "1" meaning it does.
The variable chartable is thus in a form of an 333x16 array, for 333 objects and their {0,1} value for 16 object properties as shown below:
*insert pic*
We assign the variable "indices" to the numbers 1 through 333 to be able to access each object by it's position in the table.
i.e., with indice 1, we access the first object, with indice 2 the second and so on...

In [ ]:
chartable=XLSX.readdata("Cachette_Matrix_fuer_Noor.xlsx", "Sheet1", "A2:P334");
indices=1:333;

Now we want to select from those object properties, the 7 properties that should be the generating objects of our polynomial ring. Later on we can change which object properties we want to use by switching out the integers in  "choice".
**varnum** refers to the amount of properties (i.e. the columns in the excel sheet) we select.
**choice** is an array of the integers refering to the columns in our excel sheet/chartable we want to select.
1 will select column 1 of the chartable, referring to the object property in the first column of the excel sheet
3 will select column 3 in the chartable, referring to the object property in the 3rd coloumn of the excel sheet
etc.

In [ ]:
varnum=16; choice=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]; 

<p>We reassign **chartable** to the selection of columns according to our selection in "choice". We thus get a 333:7 array of objects and their {0,1} charactarization according to the 7 selected object properties (which here are column 1, 3, 6, 19, 10, 11, 14 and 16 but can be switched out to analyze patterns and rules in a different set of object properties.)
We start building a list of object polynomials **expr_list** by ... for each object i.</p>
<p>chartable[i,:] selects the list of 0,1 xistics for object i.<br>
e.g., <br>
for the first object the following operation is performed to get its polynomial expression:<br>
prod([A,B,C,D,E,F,G]+ [A,B,C,D,E,F,G]^0 - [0,0,1,1,0,0,1]) = prod ([A,B,C,D,E,F,G]+[1,1,1,1,1,1,1]-[0,0,1,1,0,0,1]) <br>
                                                     = (A+1) * (B+1) * (C+1-1) * (D+1-1) * (E+1) * (F+1) * (G+1-1) <br>
                                                     = (A+1) * (B+1) * C * D * (E+1) * (F+1) * G <br>
We then iterate through the remainder of the objects and extend **expr_list** to get a list of all object polynomials.</p>

In [ ]:
chartable=chartable[:,choice];
expr_list=prod(myvars+ myvars.^0 -chartable[1,:]); 
for i=2:size(chartable,1)
    expr_list=vcat(expr_list, prod(myvars+ myvars.^0 -chartable[i,:])); 
end
expr_list;

We then compute the weights of the object properties. (we will need them later in the generation of the Gröbner basis)
We first create an array of ones of length 7 (varnum). We then iterate through the list of properties to print out the sum of objects that do have the property ("true="), - i.e. the value for that object in the column of said object property is 1 - and the sum of objects that do not have the property ("false=").
Finally, we calculate the weight for the specific object property by multiplying the sum of the values in the respective column by the difference between the amount of objects in total and the sum of the values in the respective column)
i.e., <br>
let sum_i = sum of values (0 or 1) for all the objects of property i <br>
    nr_of_objects = number of objects on chartable <br>
then the weight of object property i = sum_i * (nr_of_objects - sum_i)

We then add each weight to the array of ones **monweight** that we created


In [ ]:
monweight=ones(Int,varnum);
for i=1:size(myvars,1)
    print(myvars[i]);
    print(" true=");
    print(sum(chartable[:,i]));
    print(" false=");
    print(size(chartable,1)-sum(chartable[:,i]));
    print("\n");
    monweight[i]=sum(chartable[:,i])*(size(chartable,1)-sum(chartable[:,i]));
end
monweight

We now want to get the union of all polynomial expressions. Normally the union of two objects A, B is computed by A+B+(A\*B).
However, since A\*A = 0 and A+A = A; instead of creating the union of all polynomial expressions, we can also find the list of all **unique** polynomial expressions first, then just add them and get the same result. 
In that light, from our list of expressions of our objects, we now want to generate the list of unique expressions, **uq_expr_list**.
We then add these unique expressions (sum(uq_expr_list)). This union includes everything that has been observed.
To get the expression that includes everything that has **not** been observed, we need to take the complement of said union, by adding "1". <br>
(1 here is equal to the union of A,B,C,D,E,F,G))
We assign the result to variable **expr**.<br>
So far, we have created a residual ring but it still lacks the boolean property.
We begin building our generator by including all expressions that describe the boolean property, i.e. idempotency:<br>
In "myvars.^2+myvars", we generate all expressions (A\*A)+A (because if (A\*A)+A is in the ideal, then (A\*A)+A = 0 and therefore A\*A = A which is idempotency). We assign these expressions to the variable **generator** <br>
We then build our ideal **II**, of myR generated by *vcat(generator, expr)*, which is the union of everything that has not been observed and all the expressions of idempotency.<br>

quo(myR, II) will return the quotient Y of the module myR by the module II and the canonical quotient map from myR to Y.<br>

We then construct the Gröbner basis of the ideal II with respect to the ordering defined by "wdeglex((gens(myR), monweight)"<br>
wdeglex((gens(myR), monweight) returns the corresponding weighted lexicographical ordering on the set of monomials in the variables of myR.


In [ ]:
uq_expr_list=unique(expr_list);
expr=sum(uq_expr_list)+1;
generator=myvars.^2+myvars; II=ideal(myR, vcat(generator, expr));
Y, m = quo(myR, II);
GB = groebner_basis(II, ordering = wdeglex(gens(myR),monweight))

Here we are printing out ...

In [ ]:
for i = 1:size(uq_expr_list,1)
    res=normal_form(uq_expr_list[i],gens(GB));
    counter=0;
    for j = 1:size(expr_list,1)
        if (uq_expr_list[i]==expr_list[j])
            counter=counter+1;
            printstyled(indices[j], color=:red);
            print(" ");
        end
    end
    printstyled(" (#", color=:red);
    printstyled(counter, color=:red);
    printstyled(")", color=:red);
    print("\n");
    printstyled(factor(uq_expr_list[i]), color=:green);
    wrtnorm=0;
    if (wrtnorm==1)
        printstyled("=\n", color=:green);
        printstyled(uq_expr_list[i], color=:green);
    end
    print("\n");
    printstyled(res, bold=:true);
    print("\n\n");
end

In [ ]:
selection=F*B+A*B*F;
for i = 1:size(uq_expr_list,1)
    res=normal_form(uq_expr_list[i].*selection, gens(GB));
    counter=0;
    for j = 1:size(expr_list,1)
        if (uq_expr_list[i]==expr_list[j])
            counter=counter+1;
            printstyled(indices[j], color=:red);
            print(" ");
        end
    end
    printstyled(" (#", color=:red);
    printstyled(counter, color=:red);
    printstyled(")", color=:red);
    print("\n");
    printstyled(factor(uq_expr_list[i]), color=:green);
    wrtnorm=0;
    if (wrtnorm==1)
        printstyled("=\n", color=:green);
        printstyled(uq_expr_list[i], color=:green);
    end
    print("\n");
    printstyled(res, bold=:true);
    print("\n\n");
end